In [ ]:
# What is our aim?
# We are building context encoder 
# It has one encoder, followed by a channel based fully connected layer (to decrease parameters 
# and bring account distant features), and then followed by decoder (which just reverse of encoder 
# transpose coonvolutions)
# This is described above is generator which will have a L2 loss and now we will describe a discriminator
# A discriminator is something that gives output as 1 for real images and 0 for fake images
# and we are basically trying to fool our discriminator that is filling our void so visually pleasingly such that
# the disrciminator considers it real and we have an associated adversial loss with it

In [ ]:
#imports
import os
import random
import pathlib
import tensorflow as tf
from tensorflow.keras.utils import Sequence
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from tensorflow.keras import layers
from tensorflow.keras import activations



from tensorflow.keras.layers import ZeroPadding2D
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.layers import Cropping2D
from tensorflow.keras import activations
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import regularizers
from keras import backend as K
import cv2
from matplotlib import image as matimg, pyplot
import tensorflow as tf
from PIL import Image as im
from tensorflow.keras.layers import Layer
from tensorflow.keras.optimizers import Adam
import pickle
import keras.backend as K

In [ ]:
from __future__ import print_function, division
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers import MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import losses
from keras.utils import to_categorical
import keras.backend as K

import matplotlib.pyplot as plt

import numpy as np
import os

In [ ]:
## Image dataset CIFAR 10
path = open('../input/cifar10-preprocessed/data.pickle','rb')
data = pickle.load(path)
path.close

In [ ]:
# Image Dataset CIFAR 10
print(data.keys())
x_train = data['x_train']
y_train = data['y_train']
y_test = data['y_test']
x_validation = data['x_validation']
x_test = data['x_test']
y_validation = data['y_validation']

In [ ]:
# Image Dataset CIFAR 10
X_deer = x_train[(y_train==4).flatten()]
X_horse = x_train[(y_train==6).flatten()]
[R,G,B] = X_deer[13]
img = np.dstack((R,G,B))
plt.imshow(img)

In [ ]:
# ImagenetMini Dataset
X_train = np.zeros((4000,256,256,3))
i = 0;
for dirname, _, filenames in os.walk('/kaggle/input/imagenetmini-1000/imagenet-mini/train'):
        for filename in filenames:
#             print(os.path.join(dirname, filename))
            img = np.array(im.open(os.path.join(dirname, filename)))
            if(img.shape[0]>=256 and img.shape[1]>=256 and len(img.shape) == 3):
#                 print(img.shape)
                print(i)
                X_train[i,:,:,:]=img[0:256,0:256,:]
                i=i+1
                if(i==4000):
                    break
            if(i==4000):
                break
        if(i==4000):
            break
#             X_train[i]=np.array(im.open(os.path.join(dirname, filename))))

In [ ]:
#Rescale -1 to 1 image intensities
X_train = X_train/127.5 -1

In [ ]:
numrows = 256
numcols = 256
color = 3
mask_height = 64
mask_width = 64
Imshape = (numrows,numcols,color)
maskShape = (mask_height,mask_width,color)
optimizer = Adam(0.0001)

In [ ]:
modelgen = Sequential()
#           (256x256x3)
modelgen.add(Conv2D(64, kernel_size=3, strides=2, input_shape=Imshape, padding="same"))
modelgen.add(LeakyReLU(alpha=0.2))
modelgen.add(BatchNormalization(momentum=0.8))
#           (128x128x64)
modelgen.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
modelgen.add(LeakyReLU(alpha=0.2))
modelgen.add(BatchNormalization(momentum=0.8))
#           (64x64x64)
modelgen.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
modelgen.add(LeakyReLU(alpha=0.2))
modelgen.add(BatchNormalization(momentum=0.8))
#           (32x32x64)
modelgen.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
modelgen.add(LeakyReLU(alpha=0.2))
modelgen.add(BatchNormalization(momentum=0.8))
#           (16x16x128)
modelgen.add(Conv2D(256, kernel_size=3, strides=2, padding="same"))
modelgen.add(LeakyReLU(alpha=0.2))
modelgen.add(BatchNormalization(momentum=0.8))
#           (8x8x256)
modelgen.add(Conv2D(512, kernel_size=3, strides=2, padding="same"))
modelgen.add(LeakyReLU(alpha=0.2))
modelgen.add(Dropout(0.5))
#           (4x4x512)

#Decoder
modelgen.add(UpSampling2D())
#           (8x8x512)
modelgen.add(Conv2D(128, kernel_size=3, padding="same"))
#           (8x8x128)
modelgen.add(Activation('relu'))
modelgen.add(BatchNormalization(momentum=0.8))
modelgen.add(UpSampling2D())
#           (16x16x128)
modelgen.add(Conv2D(64, kernel_size=3, padding="same"))
#           (16x16x64)
modelgen.add(Activation('relu'))
modelgen.add(BatchNormalization(momentum=0.8))
modelgen.add(UpSampling2D())
#         (32x32x64)
modelgen.add(Conv2D(32, kernel_size=3, padding="same"))
#           (32x32x32)
modelgen.add(Activation('relu'))
modelgen.add(BatchNormalization(momentum=0.8))
modelgen.add(UpSampling2D())
#         (64x64x32)
modelgen.add(Conv2D(color, kernel_size=3, padding="same"))
#           (16x16x3)
modelgen.add(Activation('tanh'))

modelgen.summary()

input_img = Input(shape=Imshape)
missing = modelgen(input_img)
generator = Model(input_img,missing)

In [ ]:
#Discriminator
model = Sequential()
#           (64x64x3)
model.add(Conv2D(64, kernel_size=3, strides=2, input_shape=maskShape, padding="same"))
model.add(LeakyReLU(alpha=0.2))
model.add(BatchNormalization(momentum=0.8))
#           (32x32x32)
model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
model.add(LeakyReLU(alpha=0.2))
model.add(BatchNormalization(momentum=0.8))
#           (16x16x64)
model.add(Conv2D(256, kernel_size=3, strides=2, padding="same"))
model.add(LeakyReLU(alpha=0.2))
model.add(BatchNormalization(momentum=0.8))
#           (8x8x128)
model.add(Conv2D(512, kernel_size=3, strides=2, padding="same"))
model.add(LeakyReLU(alpha=0.2))
model.add(BatchNormalization(momentum=0.8))
#           (4x4x256)
model.add(Conv2D(512, kernel_size=3, strides=2, padding="same"))
model.add(LeakyReLU(alpha=0.2))
model.add(BatchNormalization(momentum=0.8))
#           (2x2x512)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.summary()

missing = Input(shape=maskShape)
valid = model(missing)
discriminator = Model(missing,valid)

In [ ]:
def maskImages(imgs):
        lowerY = np.random.randint(0,numrows -mask_height,imgs.shape[0])
        upperY = lowerY + mask_height
        lowerX = np.random.randint(0,numcols -mask_width,imgs.shape[0])
        upperX = lowerX + mask_width

        inputImages = np.empty_like(imgs)
        missingParts = np.empty((imgs.shape[0],mask_height,mask_width,3))
        for i in range(imgs.shape[0]):
            maskedImage = imgs[i].copy()
            missingParts[i] = maskedImage[lowerY[i]:upperY[i],lowerX[i]:upperX[i],:].copy()
            maskedImage[lowerY[i]:upperY[i],lowerX[i]:upperX[i],:] = 0
            inputImages[i] = maskedImage

        return inputImages, missingParts, (lowerY,upperY,lowerX,upperX)

In [ ]:
discriminator.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
input_img = Input(shape=Imshape)
missing = generator(input_img)
discriminator.trainable = False
valid = discriminator(missing)
combined = Model(inputs = input_img, outputs = [missing,valid])
combined.compile(loss=['mse','binary_crossentropy'], loss_weights = [0.999,0.001], optimizer=optimizer)

In [ ]:
def sample_images(epoch, imgs):
        r, c = 3, 3

        inputImages, missingParts, (y1, y2, x1, x2) = maskImages(imgs)
        generatorPredicted = generator.predict(inputImages)

#                 to bring range back to (0,1)
        imgs = 0.5 * imgs + 0.5
        inputImages = 0.5 * inputImages + 0.5
        generatorPredicted = 0.5 * generatorPredicted + 0.5

        fig, axs = plt.subplots(r, c)
        for i in range(c):
            axs[0,i].imshow(imgs[i, :,:])
            axs[0,i].axis('off')
            axs[1,i].imshow(inputImages[i, :,:])
            axs[1,i].axis('off')
            filled_in = imgs[i].copy()
            filled_in[y1[i]:y2[i], x1[i]:x2[i], :] = generatorPredicted[i]
            axs[2,i].imshow(filled_in)
            axs[2,i].axis('off')
        fig.savefig("%d.png" % epoch)
        plt.close()

In [ ]:
def train(epochs,batch_size=128, sample_interval=50):

        valid = np.ones((batch_size,1))
        fake = np.zeros((batch_size,1))

        for epoch in range(epochs):
            ids = np.random.randint(0,4000,batch_size)
            imgs = X_train[ids]

            inputImages, missingParts, _ = maskImages(imgs)

            generatorPredicted = generator.predict(inputImages)

            d_loss_real = discriminator.train_on_batch(missingParts,valid)
            d_loss_fake = discriminator.train_on_batch(generatorPredicted,fake)
            d_loss = 0.5*np.add(d_loss_fake,d_loss_real)

            g_loss = combined.train_on_batch(inputImages, [missingParts,valid])

            print ("%d [D loss: %f, acc: %.2f%%] [G loss: %f, mse: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0], g_loss[1]))

            if epoch % sample_interval == 0:
                ids = np.random.randint(0,4000,3)
                imgs = X_train[ids]
                sample_images(epoch,imgs)

In [ ]:
train(epochs=2000, batch_size=64, sample_interval = 50)

In [ ]:
trial = 990000
for i in range(0,3999,3):
    sample_images(trial+i,X_train[i:i+3])